In [13]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np
import random
import os
from sklearn.preprocessing import LabelEncoder

In [14]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42)

In [15]:
train = pd.read_parquet('./train.parquet').drop("ID", axis=1)
test = pd.read_parquet('./test.parquet').drop("ID", axis=1)
sample_submission = pd.read_csv('./dts/sample_submission.csv')

In [16]:
NaN_col = []

for i in range(len(train.columns)):
    if(sum(train[train.columns[i]].notna())<1000000):
        NaN_col.append(train.columns[i])
        
print(NaN_col)

for col in NaN_col:
    mode = train[col].mode()[0]
    train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(mode)

print('Done')

['Estimated_Departure_Time', 'Estimated_Arrival_Time', 'Origin_State', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Delay']
Done


In [17]:
qual_col = []

for i in range(len(train.dtypes)):
    if (train.dtypes[i] == 'object'):
        qual_col.append(train.columns[i])
        
print(qual_col)
qual_col.remove('Delay')
print(qual_col)

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
            
    test[i] = le.transform(test[i])
    
print('Done.')

['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number', 'Delay']
['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']
Done.


In [18]:
train = train.dropna()

column_number = {}
for i,column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
print('Done.')

Done.


In [19]:
# train_x = train.drop(columns = ['Delay','Delay_num'])
# train_y = train['Delay_num']
train = train.drop('Delay', axis=1)

In [21]:
train = TabularDataset(train)
test = TabularDataset(test)

predictor = TabularPredictor(label='Delay_num', eval_metric='log_loss').fit(train)

No path specified. Models will be saved in: "AutogluonModels\ag-20230425_060501\"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230425_060501\"
AutoGluon Version:  0.7.0
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Train Data Rows:    1000000
Train Data Columns: 17
Label Column: Delay_num
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 2]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  clas

In [22]:
# 각각의 모델의 훈련 성능을 평가할 수 있음
predictor.leaderboard(train, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist,-0.006038,-0.603787,9.404442,0.141870,3.296429,9.404442,0.141870,3.296429,1,True,2
1,KNeighborsUnif,-0.110104,-0.598417,9.277179,0.147962,3.513268,9.277179,0.147962,3.513268,1,True,1
2,RandomForestEntr,-0.110132,-0.182641,4.772781,0.128003,100.532894,4.772781,0.128003,100.532894,1,True,6
3,RandomForestGini,-0.111207,-0.181818,4.252344,0.086000,93.719473,4.252344,0.086000,93.719473,1,True,5
4,ExtraTreesGini,-0.133057,-0.181128,3.947321,0.078117,49.651039,3.947321,0.078117,49.651039,1,True,8
5,ExtraTreesEntr,-0.135861,-0.181424,3.923876,0.076902,51.852920,3.923876,0.076902,51.852920,1,True,9
6,WeightedEnsemble_L2,-0.158677,-0.179948,18.570806,0.319442,150.786274,0.135981,0.001000,0.475181,2,True,14
7,LightGBMLarge,-0.162456,-0.180184,1.991075,0.027999,6.954645,1.991075,0.027999,6.954645,1,True,13
8,LightGBM,-0.172118,-0.180297,1.842318,0.027000,6.553022,1.842318,0.027000,6.553022,1,True,4
9,XGBoost,-0.172402,-0.180535,1.200175,0.017000,10.229721,1.200175,0.017000,10.229721,1,True,11


In [23]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict_proba(test, model=model_to_use)

In [ ]:
predictor.

In [39]:
predictor2 = TabularPredictor.load('./AutogluonModels/ag-20230425_060501/')

In [41]:
predictor2.predict_proba(test)

,1,2
0,0.959033,0.040967
1,0.941749,0.058251
2,0.927629,0.072371
3,0.925577,0.074423
4,0.917388,0.082612
...,...,...
999995,0.933656,0.066344
999996,0.973910,0.026090
999997,0.944281,0.055719
999998,0.914363,0.085637


In [27]:
submission = pd.DataFrame(data=model_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('autogluon+baseline_preprocessing.csv', index=True)

In [31]:
sample_submission[['Not_Delayed','Delayed']] = model_pred

In [33]:
sample_submission.to_csv('autogluon+baseline_preprocessing.csv', index=True)

In [34]:
sample_submission.index

RangeIndex(start=0, stop=1000000, step=1)